 First, recall where this approximation H≈JTJ comes from. Let (xi,yi) be your data points, f(⋅) be your model and β be the parameters of your model. Then the objective function of the non-linear least squares problem is 12rTr where r is the vector of the residuals, ri=yi−f(xi,β). The exact Hessian of the objective function is H=JTJ+∑ri∇2ri. So the error in this approximation is H−JTJ=∑ri∇2ri. It's a good approximation when the residuals, themselves, are small; or when the 2nd derivative of the residuals is small. Linear least squares can be considered a special case where the 2nd derivative of the residuals is zero.

As for finite difference approximation, it is relatively cheap. To compute a central difference, you'll need to evaluate the Jacobian an additional 2n times (a forward difference will cost you n additional evaluations, so I wouldn't bother). The error of the central difference approximation is proportional to ∇4r and h2, where h is the step size. The optimal step size is h∼ϵ13, where ϵ is machine precision. So unless the derivatives of the residuals are blowing up, it's pretty clear that the finite difference approximation should be a LOT better. I should point out that, while the computation is minimal, the bookkeeping is nontrivial. Each finite difference on the Jacobian will give you one row of the Hessian for each residual. You'll then have to reassemble the Hessian using the formula above.

There is, however, a 3rd option. If your solver uses a Quasi-Newton method (DFP, BFGS, Bryoden, etc.), it is already approximating the Hessian at each iteration. The approximation can be quite good, as it uses the objective function and gradient values from every iteration. Most solvers will give you access to the final Hessian estimate (or its inverse). If that's an option for you, I would use that as the Hessian estimate. It's already computed and it's probably going to be a pretty good estimate.

In [1]:
import numpy as np
import crocoddyl
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numdifftools as nd

In [4]:
x = np.array([1.5, 1.5, 0.])

model = crocoddyl.ActionModelUnicycle()
T = 30
model.costWeights = np.matrix([1,1]).T

problem = crocoddyl.ShootingProblem(x.T, [ model ] * T, model)
ddp = crocoddyl.SolverDDP(problem)
ddp.solve([], [], 1000)


In [3]:
def functions(x):
    model = crocoddyl.ActionModelUnicycle()
    T = 30
    model.costWeights = np.matrix([1,1]).T

    problem = crocoddyl.ShootingProblem(x.T, [ model ] * T, model)
    ddp = crocoddyl.SolverDDP(problem)
    ddp.solve([], [], 1000)
    j = np.array(ddp.Vx)
    print(type(j[0]))
    return j[0]


In [15]:
j = ddp.Vx[0]
j = j.reshape(3, 1)

In [16]:
h = ddp.Vxx[0]
h

array([[ 11.37797708,  -2.22218284,   4.01606954],
       [ -2.22218284,  17.53326374, -11.44757606],
       [  4.01606954, -11.44757606,  20.53519374]])

In [19]:
h1 = j @ x.reshape(1, 3)
h1 = h1.T

In [20]:
h1 @ h1.T

array([[3092.05709426, 3092.05709426,    0.        ],
       [3092.05709426, 3092.05709426,    0.        ],
       [   0.        ,    0.        ,    0.        ]])

array([[ 15.81506233,  15.81506233,   0.        ],
       [ 45.60781658,  45.60781658,   0.        ],
       [-27.60195581, -27.60195581,   0.        ]])